# Download Environment Canada Weather Data for BC

This notebook can be used to browse and download hourly weather data from Environment Canada Weather Stations

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import datetime
import folium
import requests
import os, shutil

%matplotlib notebook

# Data Coverage

* Define the time period to search for and missing data threshold

In [2]:
Start = datetime.datetime.strptime('2010-01-01', '%Y-%m-%d')
End = datetime.datetime.now()
Period_Length = (End-Start)


Start_Year = Start.year
End_Year = End.year

# Proportoin of Missing Data that is Acceptable
# For a given month/year
Missing_Data_Threshold = .1

# Met Stations

Filter Data:
* Start year (optional)
    * Calculate record lengths
* Provinces

In [3]:

Provinces = ['BC']

End_Year = datetime.datetime.now().year

Stations = pd.read_csv('https://api.weather.gc.ca/collections/climate-stations/items?HAS_HOURLY_DATA=Y&f=csv&limit=10000')

Dates = ['FIRST_DATE','LAST_DATE','HLY_FIRST_DATE','HLY_LAST_DATE','DLY_FIRST_DATE','DLY_LAST_DATE',
         'MLY_FIRST_DATE','MLY_LAST_DATE']

for date in Dates:
    Stations[date] = pd.to_datetime(Stations[date])
    
Stations['HLY_REC_LENGTH'] = Stations['HLY_LAST_DATE']-Stations['HLY_FIRST_DATE']

# Stations['HLY_REC_LENGTH'] 

# Find stations that are currently collecting data in BC
Stations = Stations.loc[Stations['PROV_STATE_TERR_CODE'].isin(Provinces)].copy()

Stations = Stations.set_index(Stations['STATION_NAME'],drop = True)


Stations['STATION_NAME'].count()

175

# Webmap of Stations

In [4]:

token = "pk.eyJ1IjoianVuZXNwYWNlYm9vdHMiLCJhIjoiY2s4a285NTM1MDQwbDNocHozdXlkNzIyaSJ9.-_8gh9gG4VuprIPnmXBr3A" # your mapbox token
tileurl = 'https://api.mapbox.com/v4/mapbox.satellite/{z}/{x}/{y}@2x.png?access_token=' + str(token)

Map = folium.Map(
    location=[53,-121], zoom_start=5)#, tiles=tileurl, attr='Mapbox')

def pop_Up_Table(row):
    pop_up='''<table>
    <tr><td><b>Station Name: </b></td>><td>'''+row['STATION_NAME']+'''</td><tr>
    <tr><td><b>Start Date: </b></td>><td>'''+row['HLY_FIRST_DATE'].strftime("%d/%m/%Y")+'''</td><tr>
    <tr><td><b>End Date: </b></td>><td>'''+row['HLY_LAST_DATE'].strftime("%d/%m/%Y")+'''</td><tr>
    </table>
    '''
    return (pop_up)

for i,row in Stations.iterrows():
#     print(row['Latitude'],row['Longitude'])
    point=folium.CircleMarker(
            location=[row['y'],row['x']],
            radius=5,
            popup=i,
            fill_opacity = 1,
            fill=True,
            color='black',
            line_weight=.35,
            fill_color='blue'
    ).add_to(Map)

Map.save('Summaries/StationMap.png')

import io
from PIL import Image

# img_data = Map._to_png(5)
# img = Image.open(io.BytesIO(img_data))
# img.save('image.png')

Map

# Download Data

In [5]:


def Clear(folder):
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))
            
            
Download = False

if Download == True:
    Path = 'Data/'
    print('Cleaning ' +Path+ ' Folder')
    Clear(Path)


    for i,row in Stations.iterrows():
        Root = 'https://api.weather.gc.ca/collections/climate-hourly/items?'

        Range = 'datetime='+str(Start_Year)+'-01-01%2000:00:00/'+str(End_Year)+'-12-31%2000:00:00'

        Station = '&CLIMATE_IDENTIFIER='+str(row['CLIMATE_IDENTIFIER'])

        Last = '&sortby=LOCAL_DATE&f=csv&limit=100000&startindex=0'

        Download = (Root+Range+Station+Last)
        response = requests.get(Download)
        open(Path+str(i)+".csv", "wb").write(response.content)
        print('Downloaded: ',i)

Cleaning Data/ Folder
Downloaded:  DAWSON CREEK A
Downloaded:  WEST VANCOUVER AUT
Downloaded:  ETHELDA BAY
Downloaded:  PORT ALBERNI (AUT)
Downloaded:  SARTINE ISLAND (AUT)
Downloaded:  BLUE RIVER A
Downloaded:  PUNTZI MOUNTAIN
Downloaded:  PRINCE GEORGE
Downloaded:  REVELSTOKE A
Downloaded:  DELTA BURNS BOG
Downloaded:  PENTICTON A
Downloaded:  FORT ST. JOHN A
Downloaded:  QUESNEL A
Downloaded:  SMITHERS AIRPORT AUTO
Downloaded:  POWELL RIVER A
Downloaded:  BELLA COOLA A
Downloaded:  CAMPBELL RIVER A
Downloaded:  BELLA COOLA AIRPORT
Downloaded:  CAMPBELL RIVER A
Downloaded:  HOPE
Downloaded:  OSOYOOS CS
Downloaded:  PENTICTON A
Downloaded:  BELLA COOLA A
Downloaded:  SPARWOOD A
Downloaded:  GOLDEN A
Downloaded:  PRINCETON CS
Downloaded:  PRINCE RUPERT A
Downloaded:  PUNTZI MOUNTAIN (AUT)
Downloaded:  GREY ISLET (AUT)
Downloaded:  SOLANDER ISLAND (AUT)
Downloaded:  SMITHERS A
Downloaded:  TATLAYOKO LAKE RCS
Downloaded:  QUESNEL AWOS
Downloaded:  REVELSTOKE A
Downloaded:  HOPE A
Downloa

# VPD Function

In [6]:
                

def Calc_VPD(TA, RH):
    # Calculate vapour pressure (hPa)
    # From TA (in celsius) and RH %
    ea_H = 0.61365*np.exp((17.502*TA)/(240.97+TA))
    e_H = RH*ea_H/100
    VPD = ea_H - e_H
    VPD = np.round(VPD*10,5)     # Limit precision
    return (VPD)
            

# Parse Hourly Data

* Get Daily, Monthly, and Yearly values

In [13]:

Range = pd.date_range(Start,End,freq='H')
Data = pd.DataFrame(data = {'Obs':np.ones(Range.shape)},index=Range)
Precip=Data[['Obs']].copy()
Temp=Data[['Obs']].copy()
Wind=Data[['Obs']].copy()
RH=Data[['Obs']].copy()
AllData = {
    'PRECIP_AMOUNT':Precip,
    'TEMP':Temp,
    'WIND_SPEED':Wind,
    'RELATIVE_HUMIDITY':RH,

}

for subdir, dirs, files in os.walk(Path):
    for file in files:
        filepath = subdir + file
        try:
            SomeData = pd.read_csv(filepath,parse_dates={'Datetime':['LOCAL_DATE']},
                               low_memory=False,
                               dtype = {'CLIMATE_IDENTIFIER':str})
            SomeData = SomeData.set_index('Datetime')
            SomeData['PRECIP_AMOUNT'] = SomeData['PRECIP_AMOUNT'].astype(float)
            SomeData.loc[SomeData['PRECIP_AMOUNT']>=25,'PRECIP_AMOUNT']=np.nan
            for key,val, in AllData.items():
                Data = pd.DataFrame(data = {'Obs':np.ones(Range.shape)},index=Range)
                Data = Data.join(SomeData[key])
                Clim_ID = str(SomeData['CLIMATE_IDENTIFIER'][0])
                Data = Data.rename(columns = {key:Clim_ID})
                Year = Data.resample('Y').count()
                Year['Pct'] = Year[Clim_ID]/Year['Obs']
                AllData[key] = AllData[key].join(
                            Data.loc[
                                Data.index.year.isin(Year.loc[Year['Pct']>=(1-Missing_Data_Threshold)].index.year),
                                Clim_ID
                                    ])
        except:
            pass


AllData['VPD'] = Calc_VPD(AllData['TEMP'],AllData['RELATIVE_HUMIDITY'])

Names = {
    'PRECIP_AMOUNT':'Precpitation',
    'TEMP':'Temperature',
    'WIND_SPEED':'Wind',
    'RELATIVE_HUMIDITY':'Relative Humidity',
    'VPD':'Vapor Pressure Deficit',
}

daily = True
for key,val, in AllData.items():
    if key == 'PRECIP_AMOUNT':
        Yearly = val.resample('Y').sum().replace([0],np.nan)
        agg = 'Totals'
    else:
        Yearly = val.resample('Y').mean()
        agg = 'Averages'
#         agg = 'Averages'
    Yearly = Yearly.drop(columns='Obs')
    Yearly.index=Yearly.index.year
    Yearly = Yearly.T
    Yearly.index = Yearly.index.rename('CLIMATE_IDENTIFIER')
    if key == 'PRECIP_AMOUNT':
        Monthly = val.groupby(val.index.month).sum()
    else:
        Monthly = val.groupby(val.index.month).mean()
    Monthly= Monthly.drop(columns='Obs')
    Monthly = Monthly.T
    Monthly.index = Monthly.index.rename('CLIMATE_IDENTIFIER')
    Monthly.to_csv('Summaries/Monthly_'+Names[key]+'_'+agg+'.csv')
    Yearly.to_csv('Summaries/Yearly_'+Names[key]+'_'+agg+'.csv')
    
    if daily == True:
        
        if key == 'PRECIP_AMOUNT':
            Daily = val.loc[((val.index.year>=2021)&(val.index.year<=2022))].resample('D').sum()#.replace([0],np.nan)
            agg = 'Totals'
        else:
            Daily = val.loc[((val.index.year>=2021)&(val.index.year<=2022))].resample('D').mean()
            agg = 'Max'
            
        Daily = Daily.drop(columns='Obs')
        Daily.index = Daily.index.rename('CLIMATE_IDENTIFIER')
        Daily.to_csv('Summaries/Daily_'+Names[key]+'_'+agg+'.csv')
        
    
    
Stations.to_csv('Summaries/BC_WX_Stations.csv')
print('Done')

Done
